In [11]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def abs_sobel_thresh(img, orient='x', thresh=(0,255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output


def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output


def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def color_threshold(img, thresh1=(0, 255), thresh2=(0, 255)):
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= thresh1[0]) & (s_channel <= thresh1[1])] = 1
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= thresh2[0]) & (v_channel <= thresh2[1])] = 1
    
    bin_output = np.zeros_like(s_channel)
    bin_output[(v_binary == 1) & (s_binary == 1)] = 1


    return bin_output





In [12]:

def process_img(img):
    
    dist_pickle = pickle.load(open( "calibration_pickle/dist_pickle.p", "rb" ) )
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]

    img = cv2.undistort(img, mtx, dist, None, mtx)
    preprocessImg = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient = 'x', thresh=(12,255))
    grady = abs_sobel_thresh(img, orient = 'y', thresh=(25,255))
    cbinary = color_threshold(img, thresh1=(100,255), thresh2=(50,255))
    preprocessImg[((gradx == 1) & (grady == 1) | (cbinary == 1))] = 255
    #print(img.shape[1],img.shape[0])#1280 720
    
    
    img_size = (img.shape[1], img.shape[0])
    bot_width = .76
    mid_width = .08
    height_pct = .62              
    bot_trim = .935

    # For source points I'm grabbing the outer four detected corners
    src = np.float32([[img.shape[1]*(.5-mid_width/2), img.shape[0]*height_pct], 
                      [img.shape[1]*(.5+mid_width/2), img.shape[0]*height_pct],
                      [img.shape[1]*(.5+bot_width/2), img.shape[0]*bot_trim],
                      [img.shape[1]*(.5-bot_width/2), img.shape[0]*bot_trim]
                      
                      ])

    
    #print(src)
    offset = img_size[0]*.25
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0], 
                                 [img_size[0]-offset, img_size[1]], 
                                 [offset, img_size[1]]])
    
    #print(dst)

    # ioGiven src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(preprocessImg, M, img_size, flags=cv2.INTER_LINEAR)
    
    width = 25
    height = 80
    margin = 25
    
    
    curve_centers = tracker(Mywindow_width=width, Mywindow_height=height, Mymargin=margin, My_ym=10/720, My_xm=4/384, Mysm=15)
    
    centroids = curve_centers.find_window_centroids(warped)
    left = np.zeros_like(warped)
    right = np.zeros_like(warped)
    
    rightx = []
    leftx = []
    
    xm_per_pix = curve_centers.xm_per_pix
    ym_per_pix = curve_centers.ym_per_pix
    
    for level in range(0,len(centroids)):
        leftx.append(centroids[level][0])
        rightx.append(centroids[level][1])
        # Window_mask is a function to draw window areas
        l_mask = curve_centers.window_mask(width,height,warped,centroids[level][0],level)
        r_mask = curve_centers.window_mask(width,height,warped,centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        left[(left == 255) | ((l_mask == 1) ) ] = 255
        right[(right == 255) | ((r_mask == 1) ) ] = 255

    template = np.array(left+right,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channle 
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
    #output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

    
    ploty = range(0, warped.shape[0])
    res = np.arange(warped.shape[0] - (height/2),0,-height)
    
    left_fit = np.polyfit(res, leftx, 2)
    left_fitx = left_fit[0]*ploty*ploty + left_fit[1]*ploty + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)
    
    right_fit = np.polyfit(res, rightx, 2)
    right_fitx = right_fit[0]*ploty*ploty + right_fit[1]*ploty + right_fit[2]
    right_fitx = np.array(right_fitx, np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-width/2,left_fitx[::-1]+width/2),axis=0),np.concatenate((ploty, ploty[::-1]), axis=0))), np.int32)
    
    right_lane = np.array(list(zip(np.concatenate((right_fitx-width/2,right_fitx[::-1]+width/2),axis=0),np.concatenate((ploty, ploty[::-1]), axis=0))), np.int32)

    mid_lane = np.array(list(zip(np.concatenate((left_fitx+width/2,right_fitx[::-1]-width/2),axis=0),np.concatenate((ploty, ploty[::-1]), axis=0))), np.int32)

    road = np.zeros_like(img)
    road_bkg = np.zeros_like(img)
    
    cv2.fillPoly(road, np.int_([left_lane]), color=[188, 212, 230])
    cv2.fillPoly(road, np.int_([right_lane]), color=[0, 0, 255])
    cv2.fillPoly(road, np.int_([mid_lane]), color=[0, 255, 0])


    cv2.fillPoly(road_bkg, np.int_([left_lane]), color=[255, 255, 255])
    cv2.fillPoly(road_bkg, np.int_([right_lane]), color=[255, 255, 255])
    
    road_warped = cv2.warpPerspective(road, Minv, img_size, flags=cv2.INTER_LINEAR)
    road_warped_bkg = cv2.warpPerspective(road_bkg, Minv, img_size, flags=cv2.INTER_LINEAR)
    
    
    


    base = cv2.addWeighted(img, 1.0, road_warped_bkg, -1.0, 0.0)
    output = cv2.addWeighted(base, 1.0, road_warped, 0.7, 0.0)
    
    curve_fit_cr = np.polyfit(np.array(res, np.float32)*ym_per_pix, np.array(leftx, np.float32)*xm_per_pix,2)
    curverad = ((1+ (2*curve_fit_cr[0]*ploty[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5)/np.absolute(2*curve_fit_cr[0])
    
    cam_center = (left_fitx[-1] + right_fitx[-1])/2
    cent_diff = (cam_center - warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if cent_diff <= 0:
        side_pos = 'right'
        
    cv2.putText(output,'radius'+str(round(curverad,3))+'(m)',(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    cv2.putText(output,'vehicle is'+str(abs(round(cent_diff,3)))+'(m)'+side_pos+'off center', (50,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)


    return output
    write_name = './test_images/tracktest'+str(idx)+'.jpg'
    cv2.imwrite(write_name, output)
    

In [4]:

class tracker():
    def __init__(self, Mywindow_width, Mywindow_height, Mymargin, My_ym = 1, My_xm = 1, Mysm = 1):
        # was the line detected in the last iteration?
        self.recent_centers = []
        self.window_width = Mywindow_width
        self.window_height = Mywindow_height
        self.margin = Mymargin
        self.ym_per_pix = My_ym
        self.xm_per_pix = My_xm
        self.smooth_factor = Mysm

    def window_mask(self, width, height, img_ref, center,level):
        output = np.zeros_like(img_ref)
        output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
        return output
    

    def find_window_centroids(self, warped):
        window_width = self.window_width
        window_height = self.window_height
        margin = self.margin

        window_centroids = [] # Store the (left,right) window centroid positions per level
        window = np.ones(window_width) # Create our window template that we will use for convolutions

        # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
        # and then np.convolve the vertical image slice with the window template 

        # Sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)

        # Add what we found for the first layer
        window_centroids.append((l_center,r_center))

        # Go through each layer looking for max pixel locations
        for level in range(1,(int)(warped.shape[0]/window_height)):
            # convolve the window into the vertical slice of the image
            image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
            conv_signal = np.convolve(window, image_layer)
            # Find the best left centroid by using past left center as a reference
            # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
            offset = window_width/2
            l_min_index = int(max(l_center+offset-margin,0))
            l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
            # Find the best right centroid by using past right center as a reference
            r_min_index = int(max(r_center+offset-margin,0))
            r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
            # Add what we found for that layer
            window_centroids.append((l_center,r_center))

        self.recent_centers.append(window_centroids)

        return np.average(self.recent_centers[-self.smooth_factor:], axis = 0)


In [13]:


outputv = 'output.mp4'
inputv = 'project_video.mp4'
clip1 = VideoFileClip(inputv)
clip = clip1.fl_image(process_img)
clip.write_videofile(outputv, audio=False)

[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|█████████▉| 1260/1261 [03:40<00:00,  6.10it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 



In [10]:
#from tracker import tracker
#images = glob.glob('./test_images/test*.jpg')


#for idx, fname in enumerate(images):

#    img = cv2.imread(fname)
#    img = cv2.undistort(img,mtx,dist,None,mtx)
